# The McCall Search Model

First we load all necessary packages.

In [12]:
using LinearAlgebra

We must first create a function for the Bellman mapping.

In [30]:
"""
    mccallbellmanmap(v,w,π,c,β)

Iterates the McCall search model bellman equation for with value function v.
Returns the new value function.

# Arguments
* `v` vector of values for each wage
* `w` vector of wages
* `p` vector of probabilities for each wage
* `c` unemployment benefits
* `β` time discount factor

"""

function mccallbellmanmap(v, w, p, c, β)
    
    # create empty vector for value storage
    v_out = zeros(length(w))
    
    # compute value given each wage in vector `w`
    for i in 1:length(w)
        
        # Bellman map: v = max{c + β E(v'), w/(1-β)}
        v_out[i] = max(c + β * dot(p,v), w[i]/(1-β)) 
        
    end
    
    return v_out
end

mccallbellmanmap (generic function with 1 method)

Alternatively:

In [37]:
"""
    mccallbellmanmap_v2(v,w,π,c,β)

Iterates the McCall search model bellman equation for with value function v.
Returns the new value function.

# Arguments
* `v` vector of values for each wage
* `w` vector of wages
* `p` vector of probabilities for each wage
* `c` unemployment benefits
* `β` time discount factor

"""
function mccallbellmanmap_v2(v, w, p, c, β)
    
    # compute value of rejecting the offer
    v_reject = c + β * dot(p,v) # this is a Float
    
    # compute value of accepting the wage offer
    v_accept = w / (1-β)
    
    # compute v_reject to v_accept
    v_out = max.(v_reject, v_accept)
    
    return v_out
end

mccallbellmanmap_v2

Now let's iterate over the Bellman map to see what it does.

In [39]:
# Create discrete uniform wage distribution between 1 and 10
# where S = # of grid points (evenly-spaced points between 1 and 10)
S = 10
w = LinRange(1, 10, S) 
p = ones(S)/S # each p[i] = 1/S ∀i, so that p(w[i]) = p(w[j]) ∀i,j ∈ {1,...,S}

# Set model parameters:
β = 0.95 # note that lower β causes code to converge faster
c = 3 # unemployment wage/benefits

# Initiate the value function
v0 = zeros(S)

# Iterate code J times
J = 50

# Create and initiate array containing 
# iterated value function values:
V = zeros(J, S)
V[1,:] = mccallbellmanmap(v0, w, p, c, β)

# Keep applying Bellman map recursively 
for j in 2:J
    V[j,:] = mccallbellmanmap_v2(V[j-1,:], w, p, c, β)
end

V

50×10 Array{Float64,2}:
  20.0     40.0     60.0     80.0    …  140.0    160.0    180.0  200.0
 107.5    107.5    107.5    107.5       140.0    160.0    180.0  200.0
 130.062  130.062  130.062  130.062     140.0    160.0    180.0  200.0
 141.736  141.736  141.736  141.736     141.736  160.0    180.0  200.0
 148.554  148.554  148.554  148.554     148.554  160.0    180.0  200.0
 153.089  153.089  153.089  153.089  …  153.089  160.0    180.0  200.0
 156.104  156.104  156.104  156.104     156.104  160.0    180.0  200.0
 158.109  158.109  158.109  158.109     158.109  160.0    180.0  200.0
 159.443  159.443  159.443  159.443     159.443  160.0    180.0  200.0
 160.329  160.329  160.329  160.329     160.329  160.329  180.0  200.0
 160.95   160.95   160.95   160.95   …  160.95   160.95   180.0  200.0
 161.422  161.422  161.422  161.422     161.422  161.422  180.0  200.0
 161.781  161.781  161.781  161.781     161.781  161.781  180.0  200.0
   ⋮                                 ⋱               

Lastly, we define a function that solves any given finite-state McCall model.

In [43]:
"""
    solvemccall(w,π,c,β[,ϵ])

Iterates the McCall search model bellman equation until convergence criterion 
ϵ is reached

# Arguments
* `w` vector of wages
* `p` vector of probabilities for each wage
* `c` unemployment benefits
* `β` time discount factor
* `ϵ' Stopping criteria (default 1e-6)
"""

function solvemccall(w,p,c,β,ϵ=1e-6)
    
    # initialize the value function.
    v = w/(1-β)
    
    # initiate distance from iterated mapping
    diff = 10
    
    # keep iterating on Bellman map until
    # stopping criterium is satisfied
    while diff > ϵ
        
        # new mapping of the value function
        v_new = mccallbellmanmap(v,w,p,c,β)
        
        # use supremum norm
        # as measure of distance between 
        # old vs. new value function map
        diff = norm(v-v_new,Inf)
        
        # store new value function as original
        # before loop potentially restarts
        v = v_new 
    end
    
    # Return converged value function
    return v
end

solvemccall (generic function with 2 methods)

We can use our function to solve the McCall model defined by the previously-declared parameters. 

In [44]:
v = solvemccall(w,p,c,β)

10-element Array{Float64,1}:
 162.91666382521822
 162.91666382521822
 162.91666382521822
 162.91666382521822
 162.91666382521822
 162.91666382521822
 162.91666382521822
 162.91666382521822
 179.99999999999983
 199.99999999999983